In [178]:
import folium
import pandas as pd
import csv
from folium.plugins import MarkerCluster

In [179]:
df = pd.read_csv('서울시 보호구역2.csv', encoding='cp949')

In [180]:
df = df.drop(['Unnamed: 0'], axis=1)

In [181]:
df.head()

,시설종류,대상시설명,소재지도로명주소,위도,경도,관리기관명,CCTV설치여부,CCTV설치대수,데이터기준일자
0,초등학교,광장초등학교,서울특별시 광진구 광장로1길 27,37.548462,127.092186,서울특별시청,Y,5.0,2019-02-20
1,초등학교,구의초등학교,서울특별시 광진구 광나루로30길 49,37.542934,127.071203,서울특별시청,Y,7.0,2019-02-20
2,초등학교,동의초등학교,서울특별시 광진구 영화사로 111,37.553380,127.090046,서울특별시청,Y,7.0,2019-02-20
3,초등학교,동자초등학교,서울특별시 광진구 뚝섬로 561,37.534686,127.063360,서울특별시청,Y,3.0,2019-02-20
4,초등학교,신자초등학교,서울특별시 광진구 자양동 뚝섬로46길 64,37.529990,127.073328,서울특별시청,Y,7.0,2019-02-20


In [ ]:
seoul_risk = pd.read_csv('/content/3y_risk_index.csv', encoding='utf-8')
df = pd.read_csv('서울시 보호구역2.csv', encoding='cp949')
seoul_risk2 = seoul_risk.filter(['지역', '3year_hazard'])
seoul_risk2.columns = ['name', 'value']

In [163]:
svg = open('Seoul_districts.svg', 'r').read()

In [165]:
df_지역구 = df.groupby('지역구')['CCTV설치대수'].sum()

In [ ]:
df_무인카메라 = df.groupby('시도명').count()

In [166]:
df_지역구 = df_지역구.reset_index(drop=False)
df_지역구

,지역구,CCTV설치대수
0,강남구,222.0
1,강동구,139.0
2,강북구,145.0
3,강서구,66.0
4,관악구,165.0
5,광진구,224.0
6,금천구,45.0
7,도봉구,116.0
8,동대문구,93.0
9,서대문구,78.0


In [167]:
df_지역구.columns = ['name', 'CCTV설치대수']
df_지역구

,name,CCTV설치대수
0,강남구,222.0
1,강동구,139.0
2,강북구,145.0
3,강서구,66.0
4,관악구,165.0
5,광진구,224.0
6,금천구,45.0
7,도봉구,116.0
8,동대문구,93.0
9,서대문구,78.0


In [168]:
heatmap_df = df_지역구[['지역구', '시설종류']]

KeyError: "None of [Index(['지역구', '시설종류'], dtype='object')] are in the [columns]"

In [169]:
heatmap_df.columns = ['name', '시설개수']

In [170]:
heatmap_df

,name,시설개수
0,강남구,101
1,강동구,89
2,강북구,40
3,강서구,76
4,관악구,66
5,광진구,80
6,금천구,45
7,도봉구,70
8,동대문구,72
9,서대문구,40


In [171]:
plt_df = pd.merge(heatmap_df, df_지역구, on='name')

In [172]:
plt_df

,name,시설개수,CCTV설치대수
0,강남구,101,222.0
1,강동구,89,139.0
2,강북구,40,145.0
3,강서구,76,66.0
4,관악구,66,165.0
5,광진구,80,224.0
6,금천구,45,45.0
7,도봉구,70,116.0
8,동대문구,72,93.0
9,서대문구,40,78.0


In [173]:
plt_df.to_csv('구별 시설_CCTV 개수.csv', index=False)

In [174]:
import folium

geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [175]:
center = [37.541, 126.986] 
m = folium.Map(location=center, zoom_start=10)

In [176]:
folium.Choropleth(geo_data = geo_json,
                  data = heatmap_df,
                  name = 'choropleth',
                  columns = ['name', '시설개수'], 
                  key_on = 'feature.properties.name', 
                  fill_color = "PuRd", legend_name = "Area_Num", ).add_to(m)

In [177]:
m

In [108]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for i in range(len(df)):
        folium.CircleMarker(location=[df['위도'][i], df['경도'][i]], radius=100, color='#ffffgg',fill_color='#fffggg').add_to(marker_cluster)

m.save('map.html')

In [109]:
m

In [112]:
folium.TileLayer('cartodbdark_matter').add_to(m)
m